# Принесём сюда лучшую модель с первого задания

In [22]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.metrics import log_loss
from scipy.stats import norm

In [2]:
df = pd.read_csv('../data/data.csv')
df.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,1


In [3]:
df.date_time = pd.to_datetime(df.date_time, format='%Y-%m-%d %H:%M:%S.%f')
df['date'] = df.date_time.dt.date
df = df.sort_values(by='date_time')

In [4]:
train = df[df.date <= datetime.strptime('2021-09-30', '%Y-%m-%d').date()]
val = df[df.date == datetime.strptime('2021-10-01', '%Y-%m-%d').date()]
test = df[df.date == datetime.strptime('2021-10-02', '%Y-%m-%d').date()]

In [15]:
train_slice = len(train)
val_slice = len(val)
test_slice = len(test)

In [5]:
cols = list(set(df.columns).difference(['oaid_hash', 'banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0', 'coeff_sum1']))

In [6]:
df[cols].head()

,campaign_clicks,country_id,date,os_id,zone_id,banner_id,clicks,impressions,date_time
1390198,0,7,2021-09-01,0,30,596,0,1,2021-09-01 00:02:49
5041415,1,0,2021-09-26,3,41,29,0,1,2021-09-26 00:00:00
1442602,2,15,2021-09-26,2,1,188,0,1,2021-09-26 00:00:00
7232498,2,5,2021-09-26,2,17,52,0,1,2021-09-26 00:00:00
14938691,1,13,2021-09-26,4,47,73,0,1,2021-09-26 00:00:00


In [7]:
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder, StandardScaler

ohe = OneHotEncoder()
scaller = StandardScaler()

In [9]:
def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    #интеракции 'zone_id', 'campaign_clicks', 'country_id', 'os_id', 'impressions', 'banner_id'
    df['zone_id_banner_id'] = df['zone_id'].astype('str') + '_' + df['banner_id'].astype('str')
    df['zone_id_country_id'] = df['zone_id'].astype('str') + '_' + df['country_id'].astype('str')
    df['zone_id_os_id'] = df['zone_id'].astype('str') + '_' + df['os_id'].astype('str')
    df['country_id_os_id'] = df['country_id'].astype('str') + '_' + df['os_id'].astype('str')
    df['country_id_banner_id'] = df['country_id'].astype('str') + '_' + df['banner_id'].astype('str')
    df['os_id_banner_id'] = df['os_id'].astype('str') + '_' + df['banner_id'].astype('str')
    
    return df

In [10]:
%%time
df = feature_engineering(df)
df.head()

CPU times: user 1min 35s, sys: 6.46 s, total: 1min 42s
Wall time: 1min 42s


,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,...,coeff_sum1,impressions,clicks,date,zone_id_banner_id,zone_id_country_id,zone_id_os_id,country_id_os_id,country_id_banner_id,os_id_banner_id
1390198,2021-09-01 00:02:49,30,596,5236744527665721365,0,0,7,596,0.050,0.064560,...,-4.370191,1,0,2021-09-01,30_596,30_7,30_0,7_0,7_596,0_596
5041415,2021-09-26 00:00:00,41,29,1834033519797437404,1,3,0,29,0.002,0.016386,...,-4.898257,1,0,2021-09-26,41_29,41_0,41_3,0_3,0_29,3_29
1442602,2021-09-26 00:00:00,1,188,7416450538971744701,2,2,15,188,0.008,0.014186,...,-2.106896,1,0,2021-09-26,1_188,1_15,1_2,15_2,15_188,2_188
7232498,2021-09-26 00:00:00,17,52,1832228443297591417,2,2,5,52,0.008,0.013550,...,-3.739501,1,0,2021-09-26,17_52,17_5,17_2,5_2,5_52,2_52
14938691,2021-09-26 00:00:00,47,73,4180077124914749282,1,4,13,73,0.008,0.120974,...,-3.037939,1,0,2021-09-26,47_73,47_13,47_4,13_4,13_73,4_73


In [12]:
%%time
data = sparse.hstack([
    #оригинальные фичи
    ohe.fit_transform(df[['zone_id']]),
    ohe.fit_transform(df[['os_id']]),
    ohe.fit_transform(df[['banner_id']]),
    ohe.fit_transform(df[['country_id']]),
    sparse.csr_matrix(scaller.fit_transform(df[['campaign_clicks']])),
    #интеракции
    ohe.fit_transform(df[['zone_id_banner_id']]),
    ohe.fit_transform(df[['zone_id_country_id']]),
    ohe.fit_transform(df[['zone_id_os_id']]),
    ohe.fit_transform(df[['country_id_os_id']]),
    ohe.fit_transform(df[['country_id_banner_id']]),
    ohe.fit_transform(df[['os_id_banner_id']])
])

CPU times: user 36.4 s, sys: 4.77 s, total: 41.2 s
Wall time: 42.2 s


In [ ]:
data_train = data.tocsr()[:train_slice]
data_val = data.tocsr()[train_slice:train_slice+val_slice]
data_test = data.tocsr()[train_slice+val_slice:]

In [13]:
from sklearn.linear_model import LogisticRegression

In [16]:
%%time
clf_liblinear_final = LogisticRegression(random_state=0, 
                                     solver='liblinear',
                                     C=0.1, verbose=1
                                     ).fit(data.tocsr()[:train_slice+val_slice], df[df.date <= datetime.strptime('2021-10-01', '%Y-%m-%d').date()]['clicks'])


[LibLinear]iter  1 act 6.847e+05 pre 6.084e+05 delta 1.747e+00 f 9.491e+05 |g| 7.963e+05 CG   1
iter  2 act 8.522e+04 pre 7.062e+04 delta 1.747e+00 f 2.644e+05 |g| 1.809e+05 CG   1
cg reaches trust region boundary
iter  3 act 2.458e+04 pre 2.079e+04 delta 2.163e+00 f 1.792e+05 |g| 5.417e+04 CG   4
cg reaches trust region boundary
iter  4 act 6.682e+03 pre 5.837e+03 delta 2.606e+00 f 1.546e+05 |g| 1.777e+04 CG   4
cg reaches trust region boundary
iter  5 act 2.367e+03 pre 2.231e+03 delta 2.815e+00 f 1.479e+05 |g| 5.655e+03 CG   5
cg reaches trust region boundary
iter  6 act 1.645e+03 pre 1.614e+03 delta 3.128e+00 f 1.456e+05 |g| 3.111e+03 CG   6
cg reaches trust region boundary
iter  7 act 9.839e+02 pre 9.976e+02 delta 3.128e+00 f 1.439e+05 |g| 1.939e+03 CG   7
cg reaches trust region boundary
iter  8 act 7.724e+02 pre 7.725e+02 delta 3.214e+00 f 1.429e+05 |g| 1.520e+03 CG   9
cg reaches trust region boundary
iter  9 act 5.351e+02 pre 5.339e+02 delta 3.407e+00 f 1.422e+05 |g| 1.161e+03 

## Считаем pi_0,

In [20]:
test_pi0 = test[test.banner_id == test.banner_id0]
test_pi1 = test_pi0.copy()

gg = np.sqrt(test_pi0.g0 ** 2 + test_pi0.g1 ** 2)
cc = test_pi0.coeff_sum0 - test_pi0.coeff_sum1

In [23]:
pi_0 = norm.sf((0 - cc) / gg, cc, gg)

In [24]:
pi_0

array([0.12639264, 0.00141389, 1.        , ..., 0.        , 0.43539021,
       1.        ])

# Считаем pi_1